In [ ]:
import random
import math
import copy
import time
import logging
from typing import List, Tuple, Dict, Optional
from tabulate import tabulate

# Configure logging for debugging and tracking
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class CuttingStockProblem:
    """Class to handle the cutting stock problem logic."""

    def __init__(self, stock_length: int, piece_lengths: List[int]):
        self.stock_length = stock_length
        self.piece_lengths = piece_lengths

    def calculate_waste(self, pattern: List[int]) -> float:
        """Calculate waste for a given cutting pattern."""
        total_length = sum(self.piece_lengths[i] * pattern[i] for i in range(len(self.piece_lengths)))
        if total_length > self.stock_length:
            return float('inf')  # Invalid pattern
        return self.stock_length - total_length

    def generate_initial_pattern(self) -> List[int]:
        """Generate a valid initial cutting pattern."""
        pattern = [0] * len(self.piece_lengths)
        remaining_length = self.stock_length
        for i in range(len(self.piece_lengths)):
            max_count = remaining_length // self.piece_lengths[i]
            pattern[i] = random.randint(0, max_count)
            remaining_length -= pattern[i] * self.piece_lengths[i]
        return pattern

class Algorithm:
    """Base class for optimization algorithms."""

    @staticmethod
    def simulated_annealing(csp: CuttingStockProblem, iterations: int = 1000, initial_temp: float = 1000) \
            -> Tuple[List[int], float, int]:
        """Simulated Annealing algorithm for optimizing cutting patterns."""
        current_pattern = csp.generate_initial_pattern()
        current_waste = csp.calculate_waste(current_pattern)
        best_pattern = current_pattern[:]
        best_waste = current_waste
        temp = initial_temp
        loop_count = 0

        for _ in range(iterations):
            loop_count += 1
            new_pattern = current_pattern[:]
            idx = random.randint(0, len(csp.piece_lengths) - 1)
            change = random.choice([-1, 1])
            new_pattern[idx] = max(0, new_pattern[idx] + change)

            new_waste = csp.calculate_waste(new_pattern)
            if new_waste == float('inf'):
                continue

            delta = new_waste - current_waste
            if delta < 0 or random.random() < math.exp(-delta / temp):
                current_pattern = new_pattern[:]
                current_waste = new_waste

            if current_waste < best_waste or (current_waste == best_waste and sum(current_pattern) > sum(best_pattern)):
                best_pattern = current_pattern[:]
                best_waste = current_waste

            temp *= 0.99

        return best_pattern, best_waste, loop_count

    @staticmethod
    def hill_climbing(csp: CuttingStockProblem, iterations: int = 100) -> Tuple[List[int], float, int]:
        """Hill Climbing algorithm for optimizing cutting patterns."""
        current_pattern = csp.generate_initial_pattern()
        current_waste = csp.calculate_waste(current_pattern)
        loop_count = 0

        for _ in range(iterations):
            loop_count += 1
            neighbors = []
            for i in range(len(csp.piece_lengths)):
                for change in [-1, 1]:
                    neighbor = current_pattern[:]
                    neighbor[i] = max(0, neighbor[i] + change)
                    waste = csp.calculate_waste(neighbor)
                    if waste != float('inf'):
                        neighbors.append((neighbor, waste))

            if not neighbors:
                break

            best_neighbor, best_waste = min(neighbors, key=lambda x: x[1])
            if best_waste >= current_waste:
                break
            current_pattern, current_waste = best_neighbor, best_waste

        return current_pattern, current_waste, loop_count

    @staticmethod
    def genetic_algorithm(csp: CuttingStockProblem, population_size: int = 50, generations: int = 100) \
            -> Tuple[List[int], float, int]:
        """Genetic Algorithm for optimizing cutting patterns."""
        def create_individual():
            return csp.generate_initial_pattern()

        population = [create_individual() for _ in range(population_size)]
        loop_count = 0

        for _ in range(generations):
            loop_count += 1
            fitness = [(ind, csp.calculate_waste(ind)) for ind in population]
            population = sorted(fitness, key=lambda x: x[1])
            new_population = [ind for ind, _ in population[:population_size // 2]]

            while len(new_population) < population_size:
                parent1, parent2 = random.choice(population[:population_size // 2])[0], random.choice(population[:population_size // 2])[0]
                crossover_point = random.randint(0, len(csp.piece_lengths))
                child = parent1[:crossover_point] + parent2[crossover_point:]
                if random.random() < 0.1:  # Mutation
                    idx = random.randint(0, len(csp.piece_lengths) - 1)
                    child[idx] = max(0, child[idx] + random.choice([-1, 1]))
                if csp.calculate_waste(child) != float('inf'):
                    new_population.append(child)
                else:
                    new_population.append(create_individual())

            population = new_population

        best_individual = min(population, key=lambda x: csp.calculate_waste(x))
        return best_individual, csp.calculate_waste(best_individual), loop_count

class Agent:
    """Agent class to encapsulate algorithm execution."""

    ALGORITHM_MAP = {
        "SA": Algorithm.simulated_annealing,
        "HC": Algorithm.hill_climbing,
        "GA": Algorithm.genetic_algorithm
    }

    def __init__(self, algorithm: str):
        algorithm = algorithm.upper()  # Convert input to uppercase
        if algorithm not in self.ALGORITHM_MAP:
            raise ValueError(f"Invalid algorithm: {algorithm}. Choose from {list(self.ALGORITHM_MAP.keys())}")
        self.algorithm = algorithm

    def solve(self, csp: CuttingStockProblem) -> Tuple[List[int], float, int, float]:
        """Execute the agent's algorithm and measure runtime."""
        start_time = time.time()
        pattern, waste, loops = self.ALGORITHM_MAP[self.algorithm](csp)
        runtime = time.time() - start_time
        logging.info(f"Agent {self.algorithm} completed with waste: {waste}, loops: {loops}, runtime: {runtime:.4f}s")
        return pattern, waste, loops, runtime

class CuttingStockSolver:
    """Main solver class for the cutting stock problem."""

    def __init__(self):
        self.algorithms = ["SA", "HC", "GA"]

    def solve(self, stock_length: int, piece_lengths: List[int], num_agents: int, algorithms: List[str],
              approach_type: str) -> Tuple[List[int], float, int, float, List[Tuple]]:
        """Solve the cutting stock problem using specified approach."""
        algorithms = [alg.upper() for alg in algorithms]  # Convert all inputs to uppercase
        if not algorithms:
            raise ValueError("No algorithms provided.")
        if not all(alg in self.algorithms for alg in algorithms):
            raise ValueError(f"Invalid algorithms: {algorithms}. Choose from {self.algorithms}")

        # Assign algorithms to agents, cycling if necessary
        assigned_algorithms = []
        for i in range(num_agents):
            assigned_algorithms.append(algorithms[i % len(algorithms)])

        csp = CuttingStockProblem(stock_length, piece_lengths)
        agents = [Agent(alg) for alg in assigned_algorithms]
        best_pattern, best_waste = None, float('inf')
        total_loops, total_runtime = 0, 0
        agent_results = []

        if approach_type == "1":  # Collaborative
            for agent in agents:
                pattern, waste, loops, runtime = agent.solve(csp)
                agent_results.append((agent.algorithm, pattern, waste, loops, runtime))
                total_loops += loops
                total_runtime += runtime
                if waste < best_waste or (waste == best_waste and sum(pattern) > sum(best_pattern or pattern)):
                    best_pattern, best_waste = pattern[:], waste

            for _ in range(3):  # Additional collaborative iterations
                for agent in agents:
                    pattern, waste, loops, runtime = agent.solve(csp)
                    agent_results.append((agent.algorithm, pattern, waste, loops, runtime))
                    total_loops += loops
                    total_runtime += runtime
                    if waste < best_waste or (waste == best_waste and sum(pattern) > sum(best_pattern or pattern)):
                        best_pattern, best_waste = pattern[:], waste
        else:  # Hyper-heuristic
            for agent in agents:
                pattern, waste, loops, runtime = agent.solve(csp)
                agent_results.append((agent.algorithm, pattern, waste, loops, runtime))
                total_loops += loops
                total_runtime += runtime
                if waste < best_waste or (waste == best_waste and sum(pattern) > sum(best_pattern or pattern)):
                    best_pattern, best_waste = pattern[:], waste

        return best_pattern, best_waste, total_loops, total_runtime, agent_results

    def run_random_tests(self, num_tests: int = 10) -> None:
        """Run random tests and display results in a table."""
        table_data = []
        for i in range(num_tests):
            stock_length = random.randint(50, 200)
            num_pieces = random.randint(2, 5)
            piece_lengths = sorted([random.randint(5, min(50, stock_length // 2)) for _ in range(num_pieces)])
            num_agents = random.randint(1, 3)
            algorithms = random.sample(self.algorithms, random.randint(1, num_agents))  # Flexible algorithm count
            approach_type = random.choice(["1", "2"])

            try:
                pattern, waste, total_loops, total_runtime, _ = self.solve(
                    stock_length, piece_lengths, num_agents, algorithms, approach_type
                )
                pattern_str = ", ".join([f"{piece_lengths[j]}m x {count}" for j, count in enumerate(pattern)])

                table_data.append([
                    i + 1,
                    stock_length,
                    ", ".join(map(str, piece_lengths)),
                    num_agents,
                    ", ".join(algorithms),
                    "Collaborative" if approach_type == "1" else "Hyper",
                    pattern_str,
                    waste,
                    total_loops,
                    f"{total_runtime:.4f}s"
                ])
            except Exception as e:
                logging.error(f"Test {i+1} failed: {e}")
                continue

        headers = ["Case", "Stock Length", "Piece Lengths", "Agent Count", "Algorithms", "Approach",
                   "Cutting Pattern", "Waste", "Loop Count", "Runtime"]
        print("\nResults for 10 Random Test Cases:")
        print(tabulate(table_data, headers=headers, tablefmt="grid"))

def main():
    """Main function to handle user interaction and program flow."""
    solver = CuttingStockSolver()

    try:
        choice = input("Would you like to run random tests? (Yes/No): ").lower()
        if choice == "yes":
            solver.run_random_tests()
        else:
            stock_length = int(input("Enter stock length (e.g., 100): "))
            piece_lengths = list(map(int, input("Enter piece lengths separated by commas (e.g., 10,15,20): ").split(',')))
            num_agents = int(input("Enter number of agents (e.g., 3): "))
            print("Available algorithms: SA (Simulated Annealing), HC (Hill Climbing), GA (Genetic Algorithm)")
            print("Note: Algorithm names are case-insensitive (e.g., 'sa' or 'SA' both work)")
            print(f"Enter 1 or more algorithms (will assign to {num_agents} agents by cycling or truncating)")
            algorithms = input("Select algorithms separated by commas (e.g., SA,HC,GA): ").split(',')
            approach = input("Select approach (1: Collaborative, 2: Hyper): ")

            pattern, waste, total_loops, total_runtime, agent_results = solver.solve(
                stock_length, piece_lengths, num_agents, algorithms, approach
            )

            # Convert pattern to list of repeated piece lengths for Best Solution
            detailed_solution = []
            for i, count in enumerate(pattern):
                detailed_solution.extend([piece_lengths[i]] * count)

            print("\nDetailed Results:")
            print(f"Best Solution: {detailed_solution}")
            print("Cutting Pattern:")
            for i, count in enumerate(pattern):
                print(f"  {piece_lengths[i]} meters x {count} pieces")
            print(f"Total Waste: {waste} meters")
            print(f"Total Loop Count: {total_loops}")
            print(f"Total Runtime: {total_runtime:.4f} seconds")
            print("\nAgent Performance:")
            for alg, pat, w, loops, rt in agent_results:
                print(f"  Algorithm: {alg}")
                print(f"    Solution: {pat}")
                print(f"    Waste: {w} meters")
                print(f"    Loops: {loops}")
                print(f"    Runtime: {rt:.4f} seconds")

    except ValueError as e:
        logging.error(f"Input error: {e}")
        print("Error: Invalid input. Please ensure all inputs are valid numbers and algorithms are correct.")
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        print("An unexpected error occurred. Please try again.")

if __name__ == "__main__":
    main()

Would you like to run random tests? (Yes/No): yes

Results for 10 Random Test Cases:
+--------+----------------+--------------------+---------------+--------------+---------------+----------------------------------------------+---------+--------------+-----------+
|   Case |   Stock Length | Piece Lengths      |   Agent Count | Algorithms   | Approach      | Cutting Pattern                              |   Waste |   Loop Count | Runtime   |
+========+================+====================+===============+==============+===============+==============================================+=========+==============+===========+
|      1 |            113 | 15, 23             |             3 | GA, SA       | Hyper         | 15m x 6, 23m x 1                             |       0 |         1200 | 0.0268s   |
+--------+----------------+--------------------+---------------+--------------+---------------+----------------------------------------------+---------+--------------+-----------+
|      2 |     

In [3]:
import random
import math
import copy
import time
import logging
from typing import List, Tuple, Dict, Optional
from tabulate import tabulate

# Configure logging for debugging and tracking
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class CuttingStockProblem:
    """Class to handle the cutting stock problem logic."""

    def __init__(self, stock_length: int, piece_lengths: List[int]):
        self.stock_length = stock_length
        self.piece_lengths = piece_lengths

    def calculate_waste(self, pattern: List[int]) -> float:
        """Calculate waste for a given cutting pattern."""
        total_length = sum(self.piece_lengths[i] * pattern[i] for i in range(len(self.piece_lengths)))
        if total_length > self.stock_length:
            return float('inf')  # Invalid pattern
        return self.stock_length - total_length

    def generate_initial_pattern(self) -> List[int]:
        """Generate a valid initial cutting pattern."""
        pattern = [0] * len(self.piece_lengths)
        remaining_length = self.stock_length
        for i in range(len(self.piece_lengths)):
            max_count = remaining_length // self.piece_lengths[i]
            pattern[i] = random.randint(0, max_count)
            remaining_length -= pattern[i] * self.piece_lengths[i]
        return pattern

class Algorithm:
    """Base class for optimization algorithms."""

    @staticmethod
    def simulated_annealing(csp: CuttingStockProblem, iterations: int = 1000, initial_temp: float = 1000) \
            -> Tuple[List[int], float, int]:
        """Simulated Annealing algorithm for optimizing cutting patterns."""
        current_pattern = csp.generate_initial_pattern()
        current_waste = csp.calculate_waste(current_pattern)
        best_pattern = current_pattern[:]
        best_waste = current_waste
        temp = initial_temp
        loop_count = 0

        for _ in range(iterations):
            loop_count += 1
            new_pattern = current_pattern[:]
            idx = random.randint(0, len(csp.piece_lengths) - 1)
            change = random.choice([-1, 1])
            new_pattern[idx] = max(0, new_pattern[idx] + change)

            new_waste = csp.calculate_waste(new_pattern)
            if new_waste == float('inf'):
                continue

            delta = new_waste - current_waste
            if delta < 0 or random.random() < math.exp(-delta / temp):
                current_pattern = new_pattern[:]
                current_waste = new_waste

            if current_waste < best_waste or (current_waste == best_waste and sum(current_pattern) > sum(best_pattern)):
                best_pattern = current_pattern[:]
                best_waste = current_waste

            temp *= 0.99

        return best_pattern, best_waste, loop_count

    @staticmethod
    def hill_climbing(csp: CuttingStockProblem, iterations: int = 100) -> Tuple[List[int], float, int]:
        """Hill Climbing algorithm for optimizing cutting patterns."""
        current_pattern = csp.generate_initial_pattern()
        current_waste = csp.calculate_waste(current_pattern)
        loop_count = 0

        for _ in range(iterations):
            loop_count += 1
            neighbors = []
            for i in range(len(csp.piece_lengths)):
                for change in [-1, 1]:
                    neighbor = current_pattern[:]
                    neighbor[i] = max(0, neighbor[i] + change)
                    waste = csp.calculate_waste(neighbor)
                    if waste != float('inf'):
                        neighbors.append((neighbor, waste))

            if not neighbors:
                break

            best_neighbor, best_waste = min(neighbors, key=lambda x: x[1])
            if best_waste >= current_waste:
                break
            current_pattern, current_waste = best_neighbor, best_waste

        return current_pattern, current_waste, loop_count

    @staticmethod
    def genetic_algorithm(csp: CuttingStockProblem, population_size: int = 50, generations: int = 100) \
            -> Tuple[List[int], float, int]:
        """Genetic Algorithm for optimizing cutting patterns."""
        def create_individual():
            return csp.generate_initial_pattern()

        population = [create_individual() for _ in range(population_size)]
        loop_count = 0

        for _ in range(generations):
            loop_count += 1
            fitness = [(ind, csp.calculate_waste(ind)) for ind in population]
            population = sorted(fitness, key=lambda x: x[1])
            new_population = [ind for ind, _ in population[:population_size // 2]]

            while len(new_population) < population_size:
                parent1, parent2 = random.choice(population[:population_size // 2])[0], random.choice(population[:population_size // 2])[0]
                crossover_point = random.randint(0, len(csp.piece_lengths))
                child = parent1[:crossover_point] + parent2[crossover_point:]
                if random.random() < 0.1:  # Mutation
                    idx = random.randint(0, len(csp.piece_lengths) - 1)
                    child[idx] = max(0, child[idx] + random.choice([-1, 1]))
                if csp.calculate_waste(child) != float('inf'):
                    new_population.append(child)
                else:
                    new_population.append(create_individual())

            population = new_population

        best_individual = min(population, key=lambda x: csp.calculate_waste(x))
        return best_individual, csp.calculate_waste(best_individual), loop_count

class Agent:
    """Agent class to encapsulate algorithm execution."""

    ALGORITHM_MAP = {
        "SA": Algorithm.simulated_annealing,
        "HC": Algorithm.hill_climbing,
        "GA": Algorithm.genetic_algorithm
    }

    def __init__(self, algorithm: str):
        algorithm = algorithm.upper()  # Convert input to uppercase
        if algorithm not in self.ALGORITHM_MAP:
            raise ValueError(f"Invalid algorithm: {algorithm}. Choose from {list(self.ALGORITHM_MAP.keys())}")
        self.algorithm = algorithm

    def solve(self, csp: CuttingStockProblem) -> Tuple[List[int], float, int, float]:
        """Execute the agent's algorithm and measure runtime."""
        start_time = time.time()
        pattern, waste, loops = self.ALGORITHM_MAP[self.algorithm](csp)
        runtime = time.time() - start_time
        logging.info(f"Agent {self.algorithm} completed with waste: {waste}, loops: {loops}, runtime: {runtime:.4f}s")
        return pattern, waste, loops, runtime

class CuttingStockSolver:
    """Main solver class for the cutting stock problem."""

    def __init__(self):
        self.algorithms = ["SA", "HC", "GA"]

    def solve(self, stock_length: int, piece_lengths: List[int], num_agents: int, algorithms: List[str],
              approach_type: str) -> Tuple[List[int], float, int, float, List[Tuple]]:
        """Solve the cutting stock problem using specified approach."""
        algorithms = [alg.upper() for alg in algorithms]  # Convert all inputs to uppercase
        if not algorithms:
            raise ValueError("No algorithms provided.")
        if not all(alg in self.algorithms for alg in algorithms):
            raise ValueError(f"Invalid algorithms: {algorithms}. Choose from {self.algorithms}")

        # Assign algorithms to agents, cycling if necessary
        assigned_algorithms = []
        for i in range(num_agents):
            assigned_algorithms.append(algorithms[i % len(algorithms)])

        csp = CuttingStockProblem(stock_length, piece_lengths)
        agents = [Agent(alg) for alg in assigned_algorithms]
        best_pattern, best_waste = None, float('inf')
        total_loops, total_runtime = 0, 0
        agent_results = []

        if approach_type == "1":  # Collaborative
            for agent in agents:
                pattern, waste, loops, runtime = agent.solve(csp)
                agent_results.append((agent.algorithm, pattern, waste, loops, runtime))
                total_loops += loops
                total_runtime += runtime
                if waste < best_waste or (waste == best_waste and sum(pattern) > sum(best_pattern or pattern)):
                    best_pattern, best_waste = pattern[:], waste

            for _ in range(3):  # Additional collaborative iterations
                for agent in agents:
                    pattern, waste, loops, runtime = agent.solve(csp)
                    agent_results.append((agent.algorithm, pattern, waste, loops, runtime))
                    total_loops += loops
                    total_runtime += runtime
                    if waste < best_waste or (waste == best_waste and sum(pattern) > sum(best_pattern or pattern)):
                        best_pattern, best_waste = pattern[:], waste
        else:  # Hyper-heuristic
            for agent in agents:
                pattern, waste, loops, runtime = agent.solve(csp)
                agent_results.append((agent.algorithm, pattern, waste, loops, runtime))
                total_loops += loops
                total_runtime += runtime
                if waste < best_waste or (waste == best_waste and sum(pattern) > sum(best_pattern or pattern)):
                    best_pattern, best_waste = pattern[:], waste

        return best_pattern, best_waste, total_loops, total_runtime, agent_results

    def run_random_tests(self, num_tests: int = 10) -> None:
        """Run random tests and display results in a table."""
        table_data = []
        for i in range(num_tests):
            stock_length = random.randint(50, 200)
            num_pieces = random.randint(2, 5)
            piece_lengths = sorted([random.randint(5, min(50, stock_length // 2)) for _ in range(num_pieces)])
            num_agents = random.randint(1, 3)
            algorithms = random.sample(self.algorithms, random.randint(1, num_agents))  # Flexible algorithm count
            approach_type = random.choice(["1", "2"])

            try:
                pattern, waste, total_loops, total_runtime, _ = self.solve(
                    stock_length, piece_lengths, num_agents, algorithms, approach_type
                )
                pattern_str = ", ".join([f"{piece_lengths[j]}m x {count}" for j, count in enumerate(pattern)])

                table_data.append([
                    i + 1,
                    stock_length,
                    ", ".join(map(str, piece_lengths)),
                    num_agents,
                    ", ".join(algorithms),
                    "Collaborative" if approach_type == "1" else "Hyper",
                    pattern_str,
                    waste,
                    total_loops,
                    f"{total_runtime:.4f}s"
                ])
            except Exception as e:
                logging.error(f"Test {i+1} failed: {e}")
                continue

        headers = ["Case", "Stock Length", "Piece Lengths", "Agent Count", "Algorithms", "Approach",
                   "Cutting Pattern", "Waste", "Loop Count", "Runtime"]
        print("\nResults for 10 Random Test Cases:")
        print(tabulate(table_data, headers=headers, tablefmt="grid"))

def main():
    """Main function to handle user interaction and program flow."""
    solver = CuttingStockSolver()

    try:
        choice = input("Would you like to run random tests? (Yes/No): ").lower()
        if choice == "yes":
            solver.run_random_tests()
        else:
            stock_length = int(input("Enter stock length (e.g., 100): "))
            piece_lengths = list(map(int, input("Enter piece lengths separated by commas (e.g., 10,15,20): ").split(',')))
            num_agents = int(input("Enter number of agents (e.g., 3): "))
            print("Available algorithms: SA (Simulated Annealing), HC (Hill Climbing), GA (Genetic Algorithm)")
            print("Note: Algorithm names are case-insensitive (e.g., 'sa' or 'SA' both work)")
            print(f"Enter 1 or more algorithms (will assign to {num_agents} agents by cycling or truncating)")
            algorithms = input("Select algorithms separated by commas (e.g., SA,HC,GA): ").split(',')
            approach = input("Select approach (1: Collaborative, 2: Hyper): ")

            pattern, waste, total_loops, total_runtime, agent_results = solver.solve(
                stock_length, piece_lengths, num_agents, algorithms, approach
            )

            # Convert pattern to list of repeated piece lengths for Best Solution
            detailed_solution = []
            for i, count in enumerate(pattern):
                detailed_solution.extend([piece_lengths[i]] * count)

            print("\nDetailed Results:")
            print(f"Best Solution: {detailed_solution}")
            print("Cutting Pattern:")
            for i, count in enumerate(pattern):
                print(f"  {piece_lengths[i]} meters x {count} pieces")
            print(f"Total Waste: {waste} meters")
            print(f"Total Loop Count: {total_loops}")
            print(f"Total Runtime: {total_runtime:.4f} seconds")
            print("\nAgent Performance:")
            for alg, pat, w, loops, rt in agent_results:
                print(f"  Algorithm: {alg}")
                print(f"    Solution: {pat}")
                print(f"    Waste: {w} meters")
                print(f"    Loops: {loops}")
                print(f"    Runtime: {rt:.4f} seconds")

    except ValueError as e:
        logging.error(f"Input error: {e}")
        print("Error: Invalid input. Please ensure all inputs are valid numbers and algorithms are correct.")
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        print("An unexpected error occurred. Please try again.")

if __name__ == "__main__":
    main()

Would you like to run random tests? (Yes/No): no
Enter stock length (e.g., 100): 5496
Enter piece lengths separated by commas (e.g., 10,15,20): 19,75,23,12
Enter number of agents (e.g., 3): 4
Available algorithms: SA (Simulated Annealing), HC (Hill Climbing), GA (Genetic Algorithm)
Note: Algorithm names are case-insensitive (e.g., 'sa' or 'SA' both work)
Enter 1 or more algorithms (will assign to 4 agents by cycling or truncating)
Select algorithms separated by commas (e.g., SA,HC,GA): sa,ga
Select approach (1: Collaborative, 2: Hyper): 2

Detailed Results:
Best Solution: [19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 